# Openai格式基础调用方法

环境准备：

In [1]:
%pip install openai tiktoken

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
Note: you may need to restart the kernel to use updated packages.


## 基本格式的调用

In [2]:
from openai import OpenAI
OPENAI_BASE_URL = 'http://localhost:13000/v1'  # one-api 中转接口
client = OpenAI(
     api_key='sk-6n3IuwDfeMEZggn2A61a818dC722474292D929C27004Fe82',  # one-api token
     base_url=OPENAI_BASE_URL
)

In [3]:
response = client.chat.completions.create(
    model="qwen2:7b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
    ]
)

In [4]:
response  # 回答类

ChatCompletion(id='chatcmpl-8ac2ed14f6034c03b95e98303a2f4c49', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The 2020 World Series was won by the Los Angeles Dodgers, who defeated the Tampa Bay Rays in six games. This was the Dodgers' first World Series victory since 1988.", role='assistant', function_call=None, tool_calls=None))], created=1732000080, model='qwen2:7b', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=43, prompt_tokens=31, total_tokens=74))

In [5]:
# 查看返回给用户的答案
response.choices[0].message.content

"The 2020 World Series was won by the Los Angeles Dodgers, who defeated the Tampa Bay Rays in six games. This was the Dodgers' first World Series victory since 1988."

## role的选项

角色选项：system（系统提示）、user（用户）、assistant（智能助手回复）

## 成本计算

In [7]:
response.usage

CompletionUsage(completion_tokens=43, prompt_tokens=31, total_tokens=74)

- `prompt_tokens`：是用户提交的问题`Token`数量

- `completion_tokens`：是系统返回的答案`Token`数量

In [9]:
response.usage.completion_tokens

43

图形化展示token拆分：

![token](./img/token.png)

### 使用TikToken进行成本计算

In [16]:
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')  # 为指定的 OpenAI 模型 gpt-3.5-turbo 自动选择合适的编码器（Tokenizer）

# qwen有自己的Tokenizer，详见官方github项目

messages=[
    {"system", "You are a helpful assistant."},
    {"user",  "Who won the world series in 2020?"},
]

encoding.encode(str(messages))  # 将message转为字符串后编码，即根据词表把输入的字符串转成向量

[58,
 13922,
 2675,
 527,
 264,
 11190,
 18328,
 16045,
 364,
 9125,
 25762,
 5473,
 15546,
 2834,
 279,
 1917,
 4101,
 304,
 220,
 2366,
 15,
 50734,
 364,
 882,
 8439,
 60]

In [17]:
print(f'Prompt_Tokens: {len(encoding.encode(str(messages)))}')

Prompt_Tokens: 26


可以看到gpt的编码方式会和qwen稍有不同，因为二者的Prompt_Tokens是不一样的。

In [18]:
# 查看回复的token:

response = client.chat.completions.create(
    model="qwen2:7b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
    ]
)

result = response.choices[0].message.content
print(f'Completion_Tokens: {len(encoding.encode(str(result)))}')

Completion_Tokens: 64


实际项目中，直接使用response.usage即可，但是我们也需要知道它的底层就是tiktoken